In [5]:
import numpy as np 
import matplotlib.pyplot as plt

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
import pandas as pd

In [8]:
solar_data = pd.DataFrame(pd.read_csv('C:/Users/mhkim/OneDrive/문서/GitHub/Solar_Nbody_system/solarsystem.csv', sep=','))
solar_data.set_index('name',inplace=True,drop=True)
solar_data.head()

,distance,velocity,mu
name,,,
sun,0,0,132712440018000000000
mercury,58344000000,47400,22032000000000
venus,108160800000,35100,324859000000000
earth,149600000000,29800,398600441800000
mars,227990400000,24100,42828370000000


In [9]:
solar_data.iloc[0][2]

'132712440018000000000 '

In [10]:
palette = sns.color_palette('bright')
palette

[(0.00784313725490196, 0.24313725490196078, 1.0),
 (1.0, 0.48627450980392156, 0.0),
 (0.10196078431372549, 0.788235294117647, 0.2196078431372549),
 (0.9098039215686274, 0.0, 0.043137254901960784),
 (0.5450980392156862, 0.16862745098039217, 0.8862745098039215),
 (0.6235294117647059, 0.2823529411764706, 0.0),
 (0.9450980392156862, 0.2980392156862745, 0.7568627450980392),
 (0.6392156862745098, 0.6392156862745098, 0.6392156862745098),
 (1.0, 0.7686274509803922, 0.0),
 (0.0, 0.8431372549019608, 1.0)]

In [11]:
#(new) class Planets 


class Planet:

    def __init__ (self,PlanetName):
        __solar_data = pd.DataFrame(pd.read_csv('C:/Users/mhkim/OneDrive/문서/GitHub/Solar_Nbody_system/solarsystem.csv', sep=','))
        __solar_data.set_index('name',inplace=True,drop=True)    
        __pos = np.array([0,0],float)
        __vel = np.array([0,0],float)
        self.PlanetName = str(PlanetName)
        self.position = __pos+np.array([__solar_data.loc[self.PlanetName][0],0],float) #Nowstate position
        self.velocity = __vel+np.array([__solar_data.loc[self.PlanetName][1],0],float) #Nowstate velocity
        self.mu = __solar_data.loc[self.PlanetName][2]
        self.distance = __solar_data.loc[self.PlanetName][0] #Distance from the Sun 
    
    def update (self, new_position, new_velocity):      
        self.position += new_position
        self.velocity += new_velocity

    def info(self):
        print('Planet Name:',self.PlanetName)
        print('Distance From Sun:',self.distance)
        print('Now Coordinate:',self.position)
        print('Now Velocity:',self.velocity)
        print('Planet standard gravitational parameter (mu):',self.mu)


In [23]:
#Factoring space_objects from sun to Halley comet
for name in solar_data.index:
    globals()[name] = Planet(name)

In [ ]:
for name in solar_data.index:
    print(globals()[name].info())

In [118]:
earth = Planet('earth')
earth.info()

Planet Name: earth
Distance From Sun: 149600000000
Now Coordinate: [1.496e+11 0.000e+00]
Now Velocity: [29800.     0.]
Planet standard gravitational parameter (mu): 398600441800000 


In [114]:
newpos = np.array([0,1000],float)
newvel = np.array([0,1000],float)
earth.update(newpos,newvel)
earth.info()

Planet Name: earth
Distance From Sun: 149600000000
Now Coordinate: [1.496e+11 1.000e+03]
Now Velocity: [29800.  1000.]
Planet standard gravitational parameter (mu): 398600441800000 


In [6]:
#RK4 computation

def f(function):
    #first for sun
    coord = np.array([function[0],function[1]])
    v = np.array([function[2],function[3]])
    r0 = np.sqrt(coord[0]**2+coord[1]**2)    
    #next_step
    fr = v
    fv = -mu_value*coord/r0**3

    answer = np.array([fr[0],fr[1],fv[0],fv[1]], float)
    return answer

def rk4(function,h):
    k1 = h * f(function)
    k2 = h * f(function+k1/2)
    k3 = h * f(function+k2/2)
    k4 = h * f(function+k3)      
    return function + (k1+(2*k2)+(2*k3)+k4)/6


#이 밑에가 __main__으로 들어가야 함. 
def ARK4(functions):    
    #
    t_end =  t
    dt = 1.3e4
    end = int(t_end/dt)+1
    
    #container
    global RK4v_xlist 
    global RK4v_ylist
    global RK4v_xvlist
    global RK4v_yvlist
    global RK4v_tlist
    
    RK4v_xlist,(RK4v_ylist), (RK4v_xvlist), (RK4v_yvlist), (RK4v_tlist) = [], [], [], [],np.empty(end,float)
    RK4v_tlist[0] = 0
    
    for i in range(0,len(planet_list)):
        a,b,c,d = np.empty(end,float),np.empty(end,float),np.empty(end,float),np.empty(end,float)
        a[0],b[0],c[0],d[0] =  planet_list[i][0],planet_list[i][1],planet_list[i][2],planet_list[i][3]
        RK4v_xlist.append(a)
        RK4v_ylist.append(b) 
        RK4v_xvlist.append(c)
        RK4v_yvlist.append(d)

    #RK4 calculation 
    for i in range(end-1):
        for z in range(0,len(planet_list)):
            functions[z] = rk4(i,functions[z],RK4v_tlist[i],dt,mu_list[z])
            RK4v_xlist[z][i+1] = functions[z][0]
            RK4v_ylist[z][i+1] = functions[z][1]
            RK4v_xvlist[z][i+1] = functions[z][2]
            RK4v_yvlist[z][i+1] = functions[z][3]
        RK4v_tlist[i+1] = RK4v_tlist[i] + dt

    ans = [RK4v_xlist, RK4v_ylist, RK4v_xvlist, RK4v_yvlist, RK4v_tlist]
    return ans




In [8]:
Solsy.planets[3].PlanetName

'Earth'